# KPMG Virtual Internship
#### Case Study
**Sprocket Central Pty Ltd Data Set**<br>
<br>
**Author:** Ingrid Cadu<br>
**Last Update:** Apr, 14, 2022<br>
**Note:**<br>
The data and information in this document is reflective of a hypothetical situation and client.<br>
This document is to be used for KPMG Virtual Internship purposes only.<br>

This notebook contains the code to project KPMG task 2.<br>
- Using the existing 3 datasets (Customer demographic, customer address and transactions) as a labelled dataset, please recommend which of these 1000 new customers should be targeted to drive the most value for the organisation.
- Explain the three phases:  
    - Data Exploration
    - Model Development 
    - Interpretation.

# Libraries & Load data sets

In [1]:
#Libraries

#Data Inconsistence
import fuzzywuzzy
from fuzzywuzzy import process
import chardet

#Date data type
from pyxlsb import convert_date
from datetime import datetime
from dateutil.relativedelta import relativedelta
import calendar

#Standard
import numpy as np
import statistics
import pandas as pd

#Visualization
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly as plt
import plotly.figure_factory as ff
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import folium as flm
from pandas_profiling import ProfileReport

#geomap
from geopy.geocoders import MapQuest

#Data processing
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

#machine learning
from sklearn.cluster import KMeans
from sklearn import metrics

/home/ingrid_silva/.local/lib/python3.9/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
#Data sets

data_trans = pd.read_csv("./KPMG_VI_New_raw_data_update_final_Transactions.csv") 
data_demo = pd.read_csv("./KPMG_VI_New_raw_data_update_final_CustomerDemographic.csv")
data_add = pd.read_csv("./KPMG_VI_New_raw_data_update_final_CustomerAddress.csv")
data_newC = pd.read_csv("./KPMG_VI_New_raw_data_update_final_NewCustomerList.csv")

# Data Exploration

#### Transaction
**Summary**<br>
- change 'transaction_date' and 'product_first_sold_date' to date type
- create a new col 'loyalty'
- create col 'profit' = list_price - standard_cost
- fill the 'online_order' by 'customer_id' historical behavior
- check who these customers are and if they are the same person that purchase in Transaction_date (solution: model to classify fraud)

In [3]:
data_trans.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   transaction_id           20000 non-null  int64  
 1   product_id               20000 non-null  int64  
 2   customer_id              20000 non-null  int64  
 3   transaction_date         20000 non-null  object 
 4   online_order             19640 non-null  float64
 5   order_status             20000 non-null  object 
 6   brand                    19803 non-null  object 
 7   product_line             19803 non-null  object 
 8   product_class            19803 non-null  object 
 9   product_size             19803 non-null  object 
 10  list_price               20000 non-null  float64
 11  standard_cost            19803 non-null  float64
 12  product_first_sold_date  19803 non-null  object 
dtypes: float64(3), int64(3), object(7)
memory usage: 2.0+ MB


In [4]:
data_trans.describe()

,transaction_id,product_id,customer_id,online_order,list_price,standard_cost
count,20000.000000,20000.00000,20000.000000,19640.000000,20000.000000,19803.000000
mean,10000.500000,45.36465,1738.246050,0.500458,1107.829449,556.046951
std,5773.647028,30.75359,1011.951046,0.500013,582.825242,405.955660
min,1.000000,0.00000,1.000000,0.000000,12.010000,7.210000
25%,5000.750000,18.00000,857.750000,0.000000,575.270000,215.140000
50%,10000.500000,44.00000,1736.000000,1.000000,1163.890000,507.580000
75%,15000.250000,72.00000,2613.000000,1.000000,1635.300000,795.100000
max,20000.000000,100.00000,5034.000000,1.000000,2091.470000,1759.850000


In [5]:
data_trans.isnull().sum()

transaction_id               0
product_id                   0
customer_id                  0
transaction_date             0
online_order               360
order_status                 0
brand                      197
product_line               197
product_class              197
product_size               197
list_price                   0
standard_cost              197
product_first_sold_date    197
dtype: int64

In [6]:
data_trans.transaction_date.duplicated().sum() 

19636

In [7]:
data_trans.brand.unique()

array(['Solex', 'Trek Bicycles', 'OHM Cycles', 'Norco Bicycles',
       'Giant Bicycles', 'WeareA2B', nan], dtype=object)

In [8]:
data_trans.product_class.unique()

array(['medium', 'low', 'high', nan], dtype=object)

In [9]:
data_trans.product_size.unique()

array(['medium', 'large', 'small', nan], dtype=object)

#### Customer Demographic
**Summary**<br>
- change DOB to date type
- remove cols 'remove' & 'remo'
- create age col
- change 'M' to 'Male' & 'F' to 'Female' & see who is 'Annott'

In [10]:
data_demo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 14 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   customer_id                          4000 non-null   int64  
 1   first_name                           4000 non-null   object 
 2   last_name                            3875 non-null   object 
 3   gender                               4000 non-null   object 
 4   past_3_years_bike_related_purchases  4000 non-null   int64  
 5   DOB                                  3913 non-null   object 
 6   job_title                            3494 non-null   object 
 7   job_industry_category                3345 non-null   object 
 8   wealth_segment                       4000 non-null   object 
 9   deceased_indicator                   3999 non-null   object 
 10  owns_car                             3268 non-null   object 
 11  tenure                        

In [11]:
data_demo[['tenure','past_3_years_bike_related_purchases']].describe()

,tenure,past_3_years_bike_related_purchases
count,3912.000000,4000.000000
mean,10.656186,48.874250
std,5.658991,28.715764
min,1.000000,0.000000
25%,6.000000,24.000000
50%,11.000000,48.000000
75%,15.000000,73.000000
max,22.000000,99.000000


In [12]:
data_demo.isnull().sum()

customer_id                               0
first_name                                0
last_name                               125
gender                                    0
past_3_years_bike_related_purchases       0
DOB                                      87
job_title                               506
job_industry_category                   655
wealth_segment                            0
deceased_indicator                        1
owns_car                                732
tenure                                   88
remove                                 3821
remo                                   3757
dtype: int64

In [13]:
data_demo.gender.value_counts()

Female    2037
Male      1872
U           88
Annott       1
F            1
M            1
Name: gender, dtype: int64

In [14]:
data_demo.wealth_segment.value_counts()

Mass Customer        2000
High Net Worth       1021
Affluent Customer     979
Name: wealth_segment, dtype: int64

In [15]:
data_demo.deceased_indicator.value_counts()

N    3997
Y       2
Name: deceased_indicator, dtype: int64

In [16]:
data_demo.owns_car.value_counts()

Yes    1659
No     1605
0         4
Name: owns_car, dtype: int64

In [17]:
data_demo.job_industry_category.value_counts()

Manufacturing           799
Financial Services      774
Health                  602
Retail                  358
Property                267
IT                      223
Entertainment           136
Argiculture             113
Telecommunications       72
Programmer Analyst I      1
Name: job_industry_category, dtype: int64

#### Customer Address

In [18]:
data_add.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3999 entries, 0 to 3998
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   customer_id         3999 non-null   int64 
 1   address             3999 non-null   object
 2   postcode            3999 non-null   int64 
 3   state               3999 non-null   object
 4   country             3999 non-null   object
 5   property_valuation  3999 non-null   int64 
dtypes: int64(3), object(3)
memory usage: 187.6+ KB


In [19]:
data_add.isnull().sum()

customer_id           0
address               0
postcode              0
state                 0
country               0
property_valuation    0
dtype: int64

In [20]:
data_add.address.duplicated().sum()

3

In [21]:
data_add.state.value_counts()

NSW                2054
VIC                 939
QLD                 838
New South Wales      86
Victoria             82
Name: state, dtype: int64

#### New Customer
**Summary**
- change DOB to date type
- There are 17 without DOB - drop

In [22]:
data_newC.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 22 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   first_name                           1000 non-null   object 
 1   last_name                            971 non-null    object 
 2   gender                               1000 non-null   object 
 3   past_3_years_bike_related_purchases  1000 non-null   int64  
 4   DOB                                  983 non-null    object 
 5   job_title                            894 non-null    object 
 6   job_industry_category                835 non-null    object 
 7   wealth_segment                       1000 non-null   object 
 8   deceased_indicator                   1000 non-null   object 
 9   owns_car                             1000 non-null   object 
 10  tenure                               1000 non-null   int64  
 11  address                        

In [23]:
data_newC.isnull().sum()

first_name                               0
last_name                               29
gender                                   0
past_3_years_bike_related_purchases      0
DOB                                     17
job_title                              106
job_industry_category                  165
wealth_segment                           0
deceased_indicator                       0
owns_car                                 0
tenure                                   0
address                                  0
postcode                                 0
state                                    0
country                                  0
property_valuation                       0
Unnamed: 16                              0
Unnamed: 17                              0
Unnamed: 18                              0
Unnamed: 19                              0
Rank                                     0
Value                                    0
dtype: int64

### Report in html

In [24]:
#profile = ProfileReport(trans_ctm, title="Sprocket Central Pty Ltd Transaction Report")
#profile.to_widgets()
#profile.to_file('14042022_Sprocket_Central_Data_Report.html')

# Data Preparation

#### TRANSACTION

In [25]:
#fill the 'online_order' by 'customer_id' historical behavior

nan = data_trans[data_trans['online_order'].isnull()==True]

biz = list()
for row in nan.customer_id:
    dt = data_trans[data_trans['customer_id']==row]
    md = statistics.mode(dt['online_order'])
    biz.append(md)
    
for new_order in biz:
    data_trans['online_order'] = data_trans['online_order'].fillna(new_order)

In [26]:
#change 'transaction_date'&'product_first_sold_date' to date type

data_trans['transaction_date'] = pd.to_datetime(data_trans['transaction_date'], 
                                               format='%m/%d/%Y')
data_trans['product_first_sold_date'] = pd.to_datetime(data_trans['product_first_sold_date'], 
                                               format='%m/%d/%y')

In [27]:
#create a new col 'customer_age'

date = '04/15/22'
current = datetime.strptime(date,'%m/%d/%y')

data_trans['customer_age'] = current - data_trans['product_first_sold_date']
data_trans['customer_age'] = round(data_trans['customer_age'].dt.days/365,1)

In [28]:
#Transforming date into weekdays

bag_day = list()
for row in range(0,20000):
    bag_day.append(calendar.day_name[data_trans['transaction_date'][row].weekday()])
    
data_trans['week_day'] = bag_day

In [29]:
#creating PROFIT col

data_trans['profit'] = data_trans['list_price'] - data_trans['standard_cost']

#### DEMOGRAPHIC

In [30]:
#change DOB to date type

data_demo['DOB'] = pd.to_datetime(data_demo['DOB'], format='%Y-%m-%d')

#create age col
date = '04/15/22'
current = datetime.strptime(date,'%m/%d/%y')

data_demo['age'] = current - data_demo['DOB']
data_demo['age'] = round(data_demo['age'].dt.days/365,1)

In [31]:
#remove cols 'remove' & 'remo'

data_demo.drop(columns=['remove','remo'], inplace=True)

In [32]:
#change 'M' to 'Male' & 'F' to 'Female' & check who is 'Annott'

def replace_matches_in_column(df, column, string_to_match, min_ratio):
    # get a list of unique strings
    strings = df[column].unique()
    
    # get the top 10 closest matches to our input string
    matches = fuzzywuzzy.process.extract(string_to_match, strings, 
                                         limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio)

    # only get matches with a ratio > 90
    close_matches = [matches[0] for matches in matches if matches[1] >= min_ratio]
    print(close_matches)
    # get the rows of all the close matches in our dataframe
    rows_with_matches = df[column].isin(close_matches)

    # replace all rows with close matches with the input matches 
    df.loc[rows_with_matches, column] = string_to_match
    
    # let us know the function's done
    print("All done!")

In [33]:
#Fixing gender issues

replace_matches_in_column(df=data_demo, column='gender', string_to_match="F", min_ratio = 28)
replace_matches_in_column(df=data_demo, column='gender', string_to_match="M", min_ratio = 39)

['F', 'Female']
All done!
['M', 'Male']
All done!


In [34]:
#check who is 'Annott'?

data_demo.loc[data_demo.gender.isin(['Annott'])]

,customer_id,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,tenure,age
2346,2347,L,urette,Annott,16,1957-03-04,Assistant Manager,Programmer Analyst I,High Net Worth,NaN,NaN,11.0,65.2


#### NEW CUSTOMER

In [35]:
#Drop empty DOB

data_newC.dropna(axis=0, inplace=True)

In [36]:
#DOB to date type

data_newC['DOB'] = pd.to_datetime(data_newC['DOB'], format='%Y-%m-%d')

#create age col
date = '04/15/22'
current = datetime.strptime(date,'%m/%d/%y')

data_newC['age'] = current - data_newC['DOB']
data_newC['age'] = round(data_newC['age'].dt.days/365,1)

# Data Analysis I - Customer

The hypothesis is to test which feature is more profitable to drive to the new customers:
- Ternure
- Age
- Wealth-Segment
<br>
vs.
<br>
- Past_3_years_bike_related_purchases

In [37]:
#Merging all data sets

dt = pd.merge(data_trans,data_demo, on=['customer_id'],how='left')
data = pd.merge(dt, data_add, on=['customer_id'],how='left')

In [38]:
#Excluding 'dead' people and setting age to 100

data = data[data['deceased_indicator']=='N']

new_data = data[data['age']<=100]

## Age at first purchasing

In [74]:
#Customers by loyalty rate

colors = ['#f3f3f3',] * 6
colors[0] ='#65cd87'
colors[1] = '#edffcf'
colors[2] = '#366442'

fig = go.Figure()
fig.add_trace(go.Histogram(
    x=new_data.customer_age,
    y=new_data.profit,
    name='Target', # name used in legend and hover labels
    xbins=dict( # bins used for histogram
        start=new_data.customer_age.min(),
        end=new_data.customer_age.max(),
        size=5
    ),
    marker_color=colors,
    opacity=0.85,
    histnorm='percent',
    texttemplate="%{y:.2f}%",
    textfont=dict(family = 'Arial Black',
                  size = 24)
))
fig.update_layout(width=700, 
                  height=300, 
                  margin=dict(l=5,r=5,b=45,t=35,pad=4), 
                  paper_bgcolor="white", 
                  plot_bgcolor='white')

fig.add_annotation(text="Source: (%), Proportion of Customer's Loyalty by Profit (n = 19525)",
                    showarrow=False,
                    font=dict(size=13.5, color='gray', family='Arial'),
                    x = .30,
                    xanchor = 'center',
                    xref = 'paper',
                    y = -.15,
                    yanchor = 'bottom',
                    yref = 'paper')

fig.update_xaxes(showticklabels=False, showgrid=False)
fig.update_traces(hovertemplate="Total: %{y:.2f}% \n Age: %{x}", 
                  selector=dict(type='histogram'))
fig.update_yaxes(showticklabels=False, showgrid=False) # hide all the xticks

**Comment**:<br>
- Customers between 15 and 20 years represent ~25% of the profit along the year.
- Followed by clients between 5 and 10 years whose represent ~23%.
- And finally the third segment more profitable over the year was clients between 10 and 15 years of purchasing historical behavior.
<br>
For now on, we will set as parameter only customer age with the range of 15 and 20.5 years.

In [40]:
new_data1 = new_data[(new_data.customer_age>5.0) & (new_data.customer_age<=20.5)]

## Age

In [79]:
#Customers by age

colors = ['#f3f3f3',] * 15
colors[7] = '#edffcf' #claro
colors[6] ='#65cd87' #medio
colors[5] = '#366442' #escuro

new_data1['first_age'] = new_data1['age'] - new_data1['customer_age']

fig = go.Figure()
fig.add_trace(go.Histogram(
    x=new_data1.first_age,
    y=new_data1.past_3_years_bike_related_purchases,
    name='Target', # name used in legend and hover labels
    xbins=dict( # bins used for histogram
        start=new_data1.first_age.min(),
        end=new_data1.first_age.max(),
        size=5
    ),
    marker_color=colors,
    opacity=0.85,
    histnorm='percent',
    texttemplate="%{y:.2f}%",
    textfont=dict(family = 'Arial Black',
                  size = 24)
))
fig.update_layout(width=700, 
                  height=300, 
                  margin=dict(l=5,r=5,b=45,t=35,pad=4), 
                  paper_bgcolor="white", 
                  plot_bgcolor='white')

fig.add_annotation(text="Source: (%), Proportion of Customer's Profit by Age (n = 8401)",
                    showarrow=False,
                    font=dict(size=13.5, color='gray', family='Arial'),
                    x = .28,
                    xanchor = 'center',
                    xref = 'paper',
                    y = -.15,
                    yanchor = 'bottom',
                    yref = 'paper')

fig.update_xaxes(showticklabels=False, showgrid=False)
fig.update_traces(hovertemplate="Total: %{y:.2f}% \n Age: %{x}", 
                  selector=dict(type='histogram'))
fig.update_yaxes(showticklabels=False, showgrid=False) # hide all the xticks

**Comment**:<br>
- People whose the first purchase of the range from 25 & 40 years old represent ~40% of profit over the last three years.
- Teens age represents ~18% of past 3 years purchases, followed by old age ~17%.

## Tenure

In [84]:
#Customers by tenure

colors = ['#f3f3f3',] * 5
colors[0] = '#edffcf' #claro
colors[2] ='#65cd87' #medio
colors[1] = '#366442' #escuro

fig = go.Figure()
fig.add_trace(go.Histogram(
    x=new_data1.tenure,
    y=new_data1.past_3_years_bike_related_purchases,
    name='Target', # name used in legend and hover labels
    xbins=dict( # bins used for histogram
        start=new_data1.tenure.min(),
        end=new_data1.tenure.max(),
        size=5
    ),
    marker_color=colors,
    opacity=0.85,
    histnorm='percent',
    texttemplate="%{y:.2f}%",
    textfont=dict(family = 'Arial Black',
                  size = 24)
))
fig.update_layout(width=700, 
                  height=300, 
                  margin=dict(l=5,r=5,b=45,t=35,pad=4), 
                  paper_bgcolor="white", 
                  plot_bgcolor='white')

fig.add_annotation(text="Source: (%), Proportion of Customer's Profit by Tenure (n = 8401)",
                    showarrow=False,
                    font=dict(size=13.5, color='gray', family='Arial'),
                    x = .29,
                    xanchor = 'center',
                    xref = 'paper',
                    y = -.15,
                    yanchor = 'bottom',
                    yref = 'paper')

fig.update_xaxes(showticklabels=False, showgrid=False)
fig.update_traces(hovertemplate="Total: %{y:.2f}% \n Age: %{x}", 
                  selector=dict(type='histogram'))
fig.update_yaxes(showticklabels=False, showgrid=False) # hide all the xticks

**Comment**:
- The customers between 6 and 20 years of job career presented be more profitable this year.

## Wealth-Segmentation

In [91]:
#Customers by wealth

line = list()
def segment(df,name):
    for row in name:
        dt = df[df['wealth_segment']==row]
        line.append((dt.past_3_years_bike_related_purchases.sum()/415086.0)*100)
    return line


list_wealth = segment(new_data1,['Mass Customer', 'High Net Worth','Affluent Customer'])

law = (new_data1.wealth_segment.value_counts()/8401)*100

#colors
cool = ['#f3f3f3',] * 3
cool[0] = '#366442' #escuro

#Plots
fig = make_subplots(rows=1, 
                    cols=2, 
                    shared_yaxes=False, 
                    shared_xaxes=False)

fig.add_trace(go.Bar(x=law.index, y=law.values,
              marker_color= cool,
              opacity=0.85,
              texttemplate="%{y:.2f}%",
              textfont=dict(family = 'Arial Black',
                            size = 24))
              ,1, 1)

fig.add_trace(go.Bar(x=['Mass Customer', 'High Net Worth','Affluent Customer'], 
                     y=list_wealth,
              marker_color=cool,
              opacity=0.85,
              texttemplate="%{y:.2f}%",
              textfont=dict(family = 'Arial Black',
                            size = 24))
              ,1, 2)

fig.update_layout(width=700, 
                  height=300, 
                  margin=dict(l=5,r=5,b=70,t=35,pad=4), 
                  paper_bgcolor="white", 
                  plot_bgcolor='white',
                 showlegend=False)

fig.add_annotation(text="Source:<br>Fig.1: (%), Proportion of Wealth Segmentation (n = 8401)<br>Fig.2: (%), Proportion of Wealth Segment by Profit (n=8401)",
                    align='left',
                   showarrow=False,
                    font=dict(size=13.5, color='gray', family='Arial'),
                    x = .28,
                    xanchor = 'center',
                    xref = 'paper',
                    y = -.33,
                    yanchor = 'bottom',
                    yref = 'paper')

fig.update_xaxes(showticklabels=False, showgrid=False)
fig.update_traces(hovertemplate="Total: %{y:.2f}% \n Segment: %{x}", 
                  selector=dict(type='bar'))
fig.update_yaxes(showticklabels=False, showgrid=False) # hide all the xticks

**Comment**
- The Mass represents ~50% of the customers and profit over the year.
- The other two segments represent ~25% each one, and High Net was more profitable than Affluent.

# Data Analysis I - New Customer

In [44]:
#dropping dead people

data_newC1 = data_newC[data_newC['deceased_indicator']=='N']
len(data_newC1)

715

In [71]:
#First Filter: Age (25 & 40)

#Customers by age

colors = ['#f3f3f3',] * 15
colors[3] = '#edffcf' #claro
colors[2] ='#65cd87' #medio
colors[1] = '#366442' #escuro


fig = go.Figure()
fig.add_trace(go.Histogram(
    x=data_newC1.age,
    y=data_newC1.past_3_years_bike_related_purchases,
    name='Target', # name used in legend and hover labels
    xbins=dict( # bins used for histogram
        start=data_newC1.age.min(),
        end=data_newC1.age.max(),
        size=5
    ),
    marker_color=colors,
    opacity=0.85,
    histnorm='percent',
    texttemplate="%{y:.2f}%",
    textfont=dict(family = 'Arial Black',
                  size = 24)
))
fig.update_layout(width=700, 
                  height=300, 
                  margin=dict(l=5,r=5,b=45,t=35,pad=4), 
                  paper_bgcolor="white", 
                  plot_bgcolor='white')

fig.add_annotation(text="Source: (%), Proportion of New Customer's Profit by three years Purchasing (n = 715)",
                    showarrow=False,
                    font=dict(size=13.5, color='gray', family='Arial'),
                    x = .38,
                    xanchor = 'center',
                    xref = 'paper',
                    y = -.15,
                    yanchor = 'bottom',
                    yref = 'paper')

fig.update_xaxes(showticklabels=False, showgrid=False)
fig.update_traces(hovertemplate="Total: %{y:.2f}% \n Age: %{x}", 
                  selector=dict(type='histogram'))
fig.update_yaxes(showticklabels=False, showgrid=False) # hide all the xticks

In [46]:
#Second Filter: Tenure

colors = ['#f3f3f3',] * 5
colors[1] = '#366442' #escuro

age_tenure = data_newC1[(data_newC1['age']>25.0)&(data_newC1['age']<=40.5)]


fig = go.Figure()
fig.add_trace(go.Histogram(
    x=age_tenure.tenure,
    y=age_tenure.past_3_years_bike_related_purchases,
    name='Target', # name used in legend and hover labels
    xbins=dict( # bins used for histogram
        start=age_tenure.tenure.min(),
        end=age_tenure.tenure.max(),
        size=5
    ),
    marker_color=colors,
    opacity=0.85,
    histnorm='percent',
    texttemplate="%{y:.2f}%",
    textfont=dict(family = 'Arial Black',
                  size = 24)
))
fig.update_layout(width=700, 
                  height=300, 
                  margin=dict(l=5,r=5,b=45,t=35,pad=4), 
                  paper_bgcolor="white", 
                  plot_bgcolor='white')

fig.add_annotation(text="Source: (%), Proportion of New Customer's Profit by three years Purchasing (n = 191)",
                    showarrow=False,
                    font=dict(size=13.5, color='gray', family='Arial'),
                    x = .38,
                    xanchor = 'center',
                    xref = 'paper',
                    y = -.15,
                    yanchor = 'bottom',
                    yref = 'paper')

fig.update_xaxes(showticklabels=False, showgrid=False)
fig.update_traces(hovertemplate="Total: %{y:.2f}% \n Age: %{x}", 
                  selector=dict(type='histogram'))
fig.update_yaxes(showticklabels=False, showgrid=False) # hide all the xticks

In [96]:
#Customers by wealth segment

colors = ['#f3f3f3',] * 5
colors[3] = '#edffcf' #claro
colors[2] ='#65cd87' #medio
colors[1] = '#366442' #escuro

alpha = age_tenure[(age_tenure['tenure']>6.0)&(age_tenure['tenure']<=20.5)]
lex = alpha.wealth_segment.value_counts()

fig = go.Figure()
fig.add_trace(go.Bar(x=lex.index, y=lex.values,
              marker_color= cool,
              opacity=0.85,
              texttemplate="%{y}",
              textfont=dict(family = 'Arial Black',
                            size = 24)
                    ))
fig.update_layout(width=700, 
                  height=300, 
                  margin=dict(l=5,r=5,b=45,t=35,pad=4), 
                  paper_bgcolor="white", 
                  plot_bgcolor='white')

fig.add_annotation(text="Source: Proportion of New Customer's by Segment (n = 95)",
                    showarrow=False,
                    font=dict(size=13.5, color='gray', family='Arial'),
                    x = .29,
                    xanchor = 'center',
                    xref = 'paper',
                    y = -.15,
                    yanchor = 'bottom',
                    yref = 'paper')

fig.update_xaxes(showticklabels=False, showgrid=False)
fig.update_traces(hovertemplate="Total: %{y} \n Segment: %{x}", 
                  selector=dict(type='bar'))
fig.update_yaxes(showticklabels=False, showgrid=False) # hide all the xticks

### Conclusion - Data Analysis I - Loyalty Value

Applying the same trends found in loyal customers, the new customers with the same pattern represent ~10.76% of the data set (n=715).

# Model I - Clustering Customers

The features used will be:
- Age
- Tenure
- Wealth Segment
<br>
vs.
<br>
- Profit

## Elbow Method - Featuring variables

In [48]:
#Featuring numeric data

ss = StandardScaler()

pf = ss.fit_transform(new_data[['list_price']])
ag = ss.fit_transform(new_data[['age']])
tn = ss.fit_transform(new_data[['tenure']])

#Featuring objet data
wt = LabelEncoder().fit_transform(new_data['wealth_segment'])

new_data['age_SS'] = ag
new_data['tenure_SS'] = tn
new_data['list_price_SS'] = pf
new_data['wealth_SS'] = wt

feature = {'age': ag.reshape(-1), 
           'tenure': tn.reshape(-1),
           'wealth': wt.reshape(-1), 
           'profit': pf.reshape(-1)}

features = pd.DataFrame.from_dict(feature)
features.dropna(axis=0, inplace=True)

In [49]:
#Cluster - Age vs. profit (k=3)

ft1 = features.loc[:,['age','profit']].values

wcss1 = list()
for row in range(1,11):
    kmeans = KMeans(n_clusters=row, init='k-means++')
    kmeans.fit(ft1)
    wcss1.append(kmeans.inertia_)


plt.plot(range(1,11), wcss1, linewidth=2, color='#366442', marker="8")
plt.xlabel('K Value')
plt.ylabel('Wcss')
plt

<module 'matplotlib.pyplot' from '/usr/lib/python3/dist-packages/matplotlib/pyplot.py'>

In [50]:
#Cluster - Ternure vs. profit (k=3)

ft2 = features.loc[:,['tenure','profit']].values

wcss2 = list()
for row in range(1,11):
    kmeans = KMeans(n_clusters=row, init='k-means++')
    kmeans.fit(ft2)
    wcss2.append(kmeans.inertia_)
    

plt.plot(range(1,11), wcss2, linewidth=2, color='#366442', marker="8")
plt.xlabel('K Value')
plt.ylabel('Wcss')
plt

<module 'matplotlib.pyplot' from '/usr/lib/python3/dist-packages/matplotlib/pyplot.py'>

In [51]:
#Cluster - Wealth-Segment vs. profit (k=4)
ft3 = features.loc[:,['wealth','profit']].values

wcss3 = list()
for row in range(1,11):
    kmeans = KMeans(n_clusters=row, init='k-means++')
    kmeans.fit(ft3)
    wcss3.append(kmeans.inertia_)


plt.plot(range(1,11), wcss3, linewidth=2, color='#366442', marker="8")
plt.xlabel('K Value')
plt.ylabel('Wcss')
plt

<module 'matplotlib.pyplot' from '/usr/lib/python3/dist-packages/matplotlib/pyplot.py'>

In [52]:
#Cluster - Both vs. profit (k=3)

wcss4 = list()
for row in range(1,11):
    kmeans = KMeans(n_clusters=row, init='k-means++')
    kmeans.fit(features)
    wcss4.append(kmeans.inertia_)


plt.plot(range(1,11), wcss4, linewidth=2, color='#366442', marker="8")
plt.xlabel('K Value')
plt.ylabel('Wcss')
plt

<module 'matplotlib.pyplot' from '/usr/lib/python3/dist-packages/matplotlib/pyplot.py'>

## K-Means (Models)

In [53]:
#Setting the model - Age
KM = KMeans(n_clusters=3, random_state=9)
label = KM.fit_predict(ft1)

#plotting
plt.figure(figsize=(10,6))
plt.scatter(ft1[:,0], ft1[:,1], c=KM.labels_, cmap='rainbow')
plt.scatter(KM.cluster_centers_[:,0], KM.cluster_centers_[:,1], color='black')
plt.xlabel('Age')
plt.ylabel('Profit ($)')
plt

<module 'matplotlib.pyplot' from '/usr/lib/python3/dist-packages/matplotlib/pyplot.py'>

In [81]:
#Visualizing the data - Age

new_age = new_data[(new_data['age_SS']>-0.5)&(new_data['list_price']>0)]

#Plotting
colors = ['#f3f3f3',] * 6
colors[0] ='#65cd87'
colors[1] = '#edffcf'
colors[2] = '#366442'

fig = go.Figure()
fig.add_trace(go.Histogram(
    x=new_age.age,
    y=new_age.list_price,
    name='Target', # name used in legend and hover labels
    xbins=dict( # bins used for histogram
        start=new_age.age.min(),
        end=new_age.age.max(),
        size=5
    ),
    marker_color=colors,
    opacity=0.85,
    histnorm='percent',
    texttemplate="%{y:.2f}%",
    textfont=dict(family = 'Arial Black',
                  size = 24)
))
fig.update_layout(width=700, 
                  height=300, 
                  margin=dict(l=5,r=5,b=45,t=35,pad=4), 
                  paper_bgcolor="white", 
                  plot_bgcolor='white')

fig.add_annotation(text="Source: (%), Proportion of K-Means Customer Age (n = 13349)",
                    showarrow=False,
                    font=dict(size=13.5, color='gray', family='Arial'),
                    x = .30,
                    xanchor = 'center',
                    xref = 'paper',
                    y = -.15,
                    yanchor = 'bottom',
                    yref = 'paper')

fig.update_xaxes(showticklabels=False, showgrid=False)
fig.update_traces(hovertemplate="Total: %{y:.2f}% \n Age: %{x}", 
                  selector=dict(type='histogram'))
fig.update_yaxes(showticklabels=False, showgrid=False) # hide all the xticks

In [55]:
#Setting the model - Tenure
KM1 = KMeans(n_clusters=3, random_state=9)
label = KM1.fit_predict(ft2)

#plotting
plt.figure(figsize=(10,6))
plt.scatter(ft2[:,0], ft2[:,1], c=KM1.labels_, cmap='rainbow')
plt.scatter(KM1.cluster_centers_[:,0], KM1.cluster_centers_[:,1], color='black')
plt.xlabel('Tenure')
plt.ylabel('Price List($)')
plt

<module 'matplotlib.pyplot' from '/usr/lib/python3/dist-packages/matplotlib/pyplot.py'>

In [86]:
#Visualizing the data - Tenure

new_tenure = new_data[(new_data['tenure_SS']>0)&(new_data['list_price']>0)]

#Plotting
colors = ['#f3f3f3',] * 3
colors[0] = '#366442'

fig = go.Figure()
fig.add_trace(go.Histogram(
    x=new_tenure.tenure,
    y=new_tenure.list_price,
    name='Target', # name used in legend and hover labels
    xbins=dict( # bins used for histogram
        start=new_tenure.tenure.min(),
        end=new_tenure.tenure.max(),
        size=5
    ),
    marker_color=colors,
    opacity=0.85,
    histnorm='percent',
    texttemplate="%{y:.2f}%",
    textfont=dict(family = 'Arial Black',
                  size = 24)
))
fig.update_layout(width=700, 
                  height=300, 
                  margin=dict(l=5,r=5,b=45,t=35,pad=4), 
                  paper_bgcolor="white", 
                  plot_bgcolor='white')

fig.add_annotation(text="Source: (%), Proportion of K-Means Customer Tenure (n = 9921)",
                    showarrow=False,
                    font=dict(size=13.5, color='gray', family='Arial'),
                    x = .30,
                    xanchor = 'center',
                    xref = 'paper',
                    y = -.15,
                    yanchor = 'bottom',
                    yref = 'paper')

fig.update_xaxes(showticklabels=False, showgrid=False)
fig.update_traces(hovertemplate="Total: %{y:.2f}% \n Age: %{x}", 
                  selector=dict(type='histogram'))
fig.update_yaxes(showticklabels=False, showgrid=False) # hide all the xticks

In [88]:
#Setting the model - Wealth
KM3 = KMeans(n_clusters=3, random_state=9)
label = KM3.fit_predict(ft3)

#plotting
plt.figure(figsize=(10,6))
plt.scatter(ft3[:,0], ft3[:,1], c=KM3.labels_, cmap='rainbow')
plt.scatter(KM3.cluster_centers_[:,0], KM3.cluster_centers_[:,1], color='black')
plt.xlabel('Wealth')
plt.ylabel('List Price($)')
plt

<module 'matplotlib.pyplot' from '/usr/lib/python3/dist-packages/matplotlib/pyplot.py'>

In [93]:
#Visualizing the data - Wealth Segment

new_wealth = new_data[(new_data['wealth_SS']>0.99)&(new_data['list_price_SS']>-0.5)]

#Plotting
colors = ['#f3f3f3',] * 3
colors[0] = '#366442'

lines = list()
def segment(df,name):
    for row in name:
        dt = df[df['wealth_segment']==row]
        lines.append((dt.list_price.sum()/10980203.64)*100)
    return lines


list_wealthy = segment(new_wealth,['Mass Customer', 'High Net Worth','Affluent Customer'])

fig = go.Figure()
fig.add_trace(go.Bar(x=['Mass Customer', 'High Net Worth','Affluent Customer'], 
                     y=list_wealthy,
              marker_color=cool,
              opacity=0.85,
              texttemplate="%{y:.2f}%",
              textfont=dict(family = 'Arial Black',
                            size = 24)
                    ))
fig.update_layout(width=700, 
                  height=300, 
                  margin=dict(l=5,r=5,b=45,t=35,pad=4), 
                  paper_bgcolor="white", 
                  plot_bgcolor='white')

fig.add_annotation(text="Source: (%), Proportion of K-Means Customer Wealth Segment (n = 9805)",
                    showarrow=False,
                    font=dict(size=13.5, color='gray', family='Arial'),
                    x = .35,
                    xanchor = 'center',
                    xref = 'paper',
                    y = -.15,
                    yanchor = 'bottom',
                    yref = 'paper')

fig.update_xaxes(showticklabels=False, showgrid=False)
fig.update_traces(hovertemplate="Total: %{y:.2f}% \n Segment: %{x}", 
                  selector=dict(type='bar'))
fig.update_yaxes(showticklabels=False, showgrid=False) # hide all the xticks

# Applying Model I in "New Customers"

In [98]:
#Visualizing by Wealth Segment #542

new_wealthC1 = data_newC1.loc[data_newC1['wealth_segment'].isin(['Mass Customer', 'High Net Worth'])]

new_wealthC1.wealth_segment.value_counts()

Mass Customer     362
High Net Worth    180
Name: wealth_segment, dtype: int64

In [110]:
#Visualizating new customers by wealth segment vs. age ~405 new targets
ageC1 = new_wealthC1[new_wealthC1['age']>38]

colors = ['#f3f3f3',] * 10
colors[5] = '#edffcf' #claro
colors[2] ='#65cd87' #medio
colors[1] = '#366442' #escuro

fig = go.Figure()
fig.add_trace(go.Histogram(
    x=ageC1.age,
    name='Target', # name used in legend and hover labels
    xbins=dict( # bins used for histogram
        start=ageC1.age.min(),
        end=ageC1.age.max(),
        size=5
    ),
    marker_color=colors,
    opacity=0.85,
    histnorm='percent',
    texttemplate="%{y:.2f}%",
    textfont=dict(family = 'Arial Black',
                  size = 24)
))
fig.update_layout(width=700, 
                  height=300, 
                  margin=dict(l=5,r=5,b=45,t=35,pad=4), 
                  paper_bgcolor="white", 
                  plot_bgcolor='white')

fig.add_annotation(text="Source: Proportion of New Customer's by Age (n = 542)",
                    showarrow=False,
                    font=dict(size=13.5, color='gray', family='Arial'),
                    x = .29,
                    xanchor = 'center',
                    xref = 'paper',
                    y = -.15,
                    yanchor = 'bottom',
                    yref = 'paper')

fig.update_xaxes(showticklabels=False, showgrid=False)
fig.update_traces(hovertemplate="Total: %{y} \n Age: %{x}", 
                  selector=dict(type='histogram'))
fig.update_yaxes(showticklabels=False, showgrid=False) # hide all the xticks

In [112]:
#Visualizating new customers by wealth segment&age vs. tenure ~257 new targets
tenC1 = ageC1[ageC1['tenure']>=10]

colors = ['#f3f3f3',] * 3
colors[1] ='#65cd87' #medio
colors[0] = '#366442' #escuro

fig = go.Figure()
fig.add_trace(go.Histogram(
    x=tenC1.tenure,
    name='Target', # name used in legend and hover labels
    xbins=dict( # bins used for histogram
        start=tenC1.tenure.min(),
        end=tenC1.tenure.max(),
        size=5
    ),
    marker_color=colors,
    opacity=0.85,
    histnorm='percent',
    texttemplate="%{y:.2f}%",
    textfont=dict(family = 'Arial Black',
                  size = 24)
))
fig.update_layout(width=700, 
                  height=300, 
                  margin=dict(l=5,r=5,b=45,t=35,pad=4), 
                  paper_bgcolor="white", 
                  plot_bgcolor='white')

fig.add_annotation(text="Source: Proportion of New Customer's by Tenure (n = 257)",
                    showarrow=False,
                    font=dict(size=13.5, color='gray', family='Arial'),
                    x = .29,
                    xanchor = 'center',
                    xref = 'paper',
                    y = -.15,
                    yanchor = 'bottom',
                    yref = 'paper')

fig.update_xaxes(showticklabels=False, showgrid=False)
fig.update_traces(hovertemplate="Total: %{y} \n Age: %{x}", 
                  selector=dict(type='histogram'))
fig.update_yaxes(showticklabels=False, showgrid=False) # hide all the xticks

# Conclusion

**Value: Loyalty**<br>
- The first approach using profit (difference between list_price and standard_cost) & past 3 years purchasing presented that young people between 25 (25 & 30) & 40 years with a tenure between 6 & 10 years, classified as 'Mass Customers' or 'High Net Customer', have the same pattern of whose clients with more than 10 years of loyalty. From the new 715 customers, 77 have the 'loyal pattern' and they represent 9.6% of the data set.

**Value: Profitability**<br>
- The second approach using list_price presented that customers higher than 38 years old with a tenure higher than 10 and classified as 'Mass Customer' or 'High Net Customer' can be the target to achieve a high profit. From the 715 new customers, 257 have the 'profitable pattern' and they represent 35.09% of the data set.

# Other Visualizations

In [62]:
#Seetting data

new_data.dropna(subset=['address'], axis=0, inplace=True)

#Geolocator
geolocator = MapQuest(api_key="BaTN20oblurw3CWdN1aHiiLYSOMh7gWM")

def local(df,col):
    lat = list()
    long = list()
    for row in df[col]:
        try:
            location = geolocator.geocode(row)
            lat.append(location.latitude)
            long.append(location.longitude)
        except:
            lat.append("no_response")
            long.append("no_response")
    v = {'latitude': lat, 'longitude': long}
    df = pd.DataFrame(data=v)
    return df

In [63]:
coord_cust_dt1 = local(new_data1,"address")

### Saving *CSV Files

In [67]:
coord_cust_dt1.to_csv("customer_address_coord.csv", index=False)